# Merge webpage info from HIFLDS data set into hospital data set

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances
from tqdm import tqdm

In [4]:
hospital_FILDS = pd.read_csv("../data/hospital_level_info/02_hospital_info.csv")
hospital_sam = pd.read_excel("../data/hospital_level_info/05_tbl_usr_rpt_5103.xlsx")
hospital_FILDS["ZIP_STR"] = hospital_FILDS["ZIP"].apply(lambda x : str(x))
hospitals_full = pd.read_csv("../intermediate/03_hospital_level_info_merged.csv", index_col = 0)

In [21]:
def get_index_closest_pt(pt, pt_set):
    distances = np.square(pt - pt_set).sum(axis = 1)
    min_index = distances.argmin()
    return min_index

def get_website(entry, hospital_FILDS):
    same_zip = hospital_FILDS[hospital_FILDS["ZIP_STR"] == entry["ZIP"]]
    if same_zip.shape[0] == 0:
        return "NOT AVAILABLE"
    elif same_zip.shape[0] == 1:
        return same_zip["WEBSITE"].iloc[0]
    else:
        min_index = get_index_closest_pt(np.array(entry[["Longitude", "Latitude"]]), np.array(same_zip[["X", "Y"]]))
        return same_zip.iloc[min_index]["WEBSITE"]

In [17]:
entry = hospital_sam.iloc[1]
hospital_FILDS[hospital_FILDS["ZIP_STR"] == entry["ZIP"]]["WEBSITE"].iloc[0]

'http://www.mmcenters.com//index.php/facilities/marshall_south'

In [19]:
get_website(hospital_sam.iloc[1], hospital_FILDS)

'http://www.mmcenters.com//index.php/facilities/marshall_south'

In [23]:
hospital_websites = []
for i in tqdm(range(hospitals_full.shape[0])):
    hospital_websites.append(get_website(hospitals_full.iloc[i], hospital_FILDS))

100%|██████████| 7143/7143 [00:20<00:00, 349.32it/s]


In [24]:
hospital_websites[:10]

['http://www.samc.org',
 'http://www.mmcenters.com//index.php/facilities/marshall_south',
 'NOT AVAILABLE',
 'http://www.mizellmh.com',
 'NOT AVAILABLE',
 'http://www.mmcenters.com',
 'http://www.stvhs.com/east/',
 'http://www.dekalbregional.com',
 'NOT AVAILABLE',
 'NOT AVAILABLE']

In [25]:
hospitals_full["Website"] = hospital_websites

In [27]:
(hospitals_full["Website"] == "NOT AVAILABLE").mean()

0.16771664566708666

In [29]:
hospitals_full.to_csv("../intermediate/04_hospital_level_info_merged_with_website.csv")

In [31]:
hospitals_full["Website"]

0                                     http://www.samc.org
1       http://www.mmcenters.com//index.php/facilities...
2                                           NOT AVAILABLE
3                                 http://www.mizellmh.com
4                                           NOT AVAILABLE
5                                http://www.mmcenters.com
6                              http://www.stvhs.com/east/
7                           http://www.dekalbregional.com
8                                           NOT AVAILABLE
9                                           NOT AVAILABLE
10                             http://www.helenkeller.com
11                             http://www.dalemedical.org
12                   http://www.cherokeemedicalcenter.com
13                                          NOT AVAILABLE
14                                 http://www.jackson.org
15                          http://www.lanierhospital.com
16                                    http://www.eamc.org
17            